In [1]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import pandas as pd
import project_libs as libs
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, ExtraTreesRegressor
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error



df = libs.read_csv_to_dataframe("data/data.csv")
df = df.drop(columns='date', axis=1)
df = libs.preprocess_missing_values(df)

# Display the first few rows of the preprocessed features
df.head()

shape (4600, 18)
----------------------------------------------------------------------------------------------------
List of columns
['date', 'price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'street', 'city', 'statezip', 'country']
----------------------------------------------------------------------------------------------------
Data info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4600 entries, 0 to 4599
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           4600 non-null   object 
 1   price          4600 non-null   float64
 2   bedrooms       4600 non-null   float64
 3   bathrooms      4600 non-null   float64
 4   sqft_living    4600 non-null   int64  
 5   sqft_lot       4600 non-null   int64  
 6   floors         4600 non-null   float64
 7   waterfront     4600 non-null   i

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated,street,city,statezip,country
0,313000.0,3.0,1.50,1340,7912,1.5,0,0,3,1340,0,1955,2005,18810 Densmore Ave N,Shoreline,WA 98133,USA
1,2384000.0,5.0,2.50,3650,9050,2.0,0,4,5,3370,280,1921,0,709 W Blaine St,Seattle,WA 98119,USA
2,342000.0,3.0,2.00,1930,11947,1.0,0,0,4,1930,0,1966,0,26206-26214 143rd Ave SE,Kent,WA 98042,USA
3,420000.0,3.0,2.25,2000,8030,1.0,0,0,4,1000,1000,1963,0,857 170th Pl NE,Bellevue,WA 98008,USA
4,550000.0,4.0,2.50,1940,10500,1.0,0,0,4,1140,800,1976,1992,9105 170th Ave NE,Redmond,WA 98052,USA


In [2]:
feature = ['city','statezip','country','street']
X_train, X_test, y_train, y_test = libs.Feature_Encoding(df, 'price', feature, encoding_method='label')

X_train.shape

Data Frame shape (4600, 17)
((3680, 16), (920, 16), (3680,), (920,))


(3680, 16)

In [3]:
# Define the list of models to run in the pipeline
models = [
    ('Random Forest', RandomForestRegressor, {'n_estimators': 100}),
    ('Gradient Boosting', GradientBoostingRegressor, {'n_estimators': 100}),
    ('Ridge Regression', Ridge, {'alpha': 1.0}),
    ('Lasso Regression', Lasso, {'alpha': 1.0}),
    ('ElasticNet Regression', ElasticNet, {'alpha': 1.0, 'l1_ratio': 0.5}),
    ('Decision Tree', DecisionTreeRegressor, {'max_depth': 5}),
    ('Extra Trees', ExtraTreesRegressor, {'n_estimators': 100}),
    ('KNN', KNeighborsRegressor, {'n_neighbors': 5}),
    ('Gaussian Process', GaussianProcessRegressor, {'n_restarts_optimizer': 10, 'alpha': 0.1}),
    
]
libs.run_ml_pipeline(X_train, X_test, y_train, y_test, models, use_cross_validation=False)

Model: Random Forest
Training Score: 0.85
Test Score: -0.02
----------------------------------------
Model: Gradient Boosting
Training Score: 0.84
Test Score: 0.29
----------------------------------------
Model: Ridge Regression
Training Score: 0.18
Test Score: 0.52
----------------------------------------
Model: Lasso Regression
Training Score: 0.18
Test Score: 0.52
----------------------------------------
Model: ElasticNet Regression
Training Score: 0.17
Test Score: 0.51
----------------------------------------
Model: Decision Tree
Training Score: 0.59
Test Score: -1.64
----------------------------------------


c:\Users\Lardex\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.894e+14, tolerance: 1.292e+11
  model = cd_fast.enet_coordinate_descent(
c:\Users\Lardex\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.356e+14, tolerance: 1.292e+11
  model = cd_fast.enet_coordinate_descent(


Model: Extra Trees
Training Score: 1.00
Test Score: -1.83
----------------------------------------
Model: KNN
Training Score: 0.33
Test Score: -0.19
----------------------------------------
Model: Gaussian Process
Training Score: 0.98
Test Score: -1.70
----------------------------------------
              Model Name  Training Score  Test Score
0          Random Forest        0.850737   -0.016491
1      Gradient Boosting        0.844953    0.291120
2       Ridge Regression        0.179321    0.524566
3       Lasso Regression        0.179323    0.524878
4  ElasticNet Regression        0.173921    0.506724
5          Decision Tree        0.590619   -1.637879
6            Extra Trees        0.999999   -1.831837
7                    KNN        0.327151   -0.193898
8       Gaussian Process        0.984621   -1.702665


In [4]:
model = RandomForestRegressor(n_estimators= 100)

model.fit(X_train, y_train)

pred = model.predict(X_test)

mean_squared_error(y_test, pred)

185387127313.15738